In [1]:
from openpyxl import Workbook
from bs4 import BeautifulSoup
import random
import string
import requests

In [2]:
wb = Workbook()
gielda = wb.create_sheet("Giełda",0)
linki = wb.create_sheet("Linki",1)
filmweb = wb.create_sheet("Filmweb",2)
wb.remove(wb['Sheet'])
wb.save('Lab4_Murawski.xlsx')

In [3]:
shortcuts = []

def shortcut(stringLength=3):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(stringLength))

def extract(company_name):
    start = "("
    end = ")"
    return company_name[company_name.find(start)+len(start):company_name.rfind(end)]

def exchange():    
    test = shortcut()
    req = requests.get("https://stooq.pl/q/?s="+test)
    req2 = requests.get("https://stooq.pl/q/s/?e="+test+"&t=")
    soup = BeautifulSoup(req.text, 'html.parser')        
    soup2 = BeautifulSoup(req2.text, 'html.parser')
    try:
        result = soup.find(text="Kurs")
        result2 = soup2.find('font',id='f16').find('i').find('a',href=True)
        if result:
            shortcuts.append(test)
        elif result2:
            r = result2.text
            shortcuts.append(extract(r))    
    except:
        print('szukam dalej')
while len(shortcuts) < 5:        
    exchange()

print(shortcuts)
gielda['A1'] = 'Symbol'
gielda['A2'] = 'Kurs'
gielda['A3'] = 'Zmiana'
gielda['A4'] = 'Transakcje'
for s in shortcuts:
    req = requests.get("https://stooq.pl/q/?s="+s)
    soup = BeautifulSoup(req.text, 'html.parser')
    i = shortcuts.index(s)
    gielda.cell(row=1,column=i+2,value=s)
    try:
        znak = soup.find(text="Kurs").parent.find("a",href=True).text
        gielda.cell(row=2, column=i+2).value = soup.find(text="Kurs").parent.find("span").text + znak
    except:
        gielda.cell(row=2, column=i+2).value = soup.find(text="Kurs").parent.find("span").text
    gielda.cell(row=3,column=i+2).value = soup.find(text="Zmiana").parent.find("span").text
    gielda.cell(row=4,column=i+2).value = soup.find(text="Transakcje").parent.find("span").text

wb.save('Lab4_Murawski.xlsx')   

szukam dalej
szukam dalej
['HEV20.F', 'XAGARS', 'ZSL.US', 'SMAR.US', 'NMFC.US']


In [4]:
links=[]
req = requests.get("http://wniuo.amw.gdynia.pl")
soup = BeautifulSoup(req.text,'html.parser')
for link in soup.findAll('a'):
    links.append(link.get('href'))
for l in links:
    i = links.index(l)
    linki.cell(row=i+1,column=1).value = l
wb.save('Lab4_Murawski.xlsx')

In [5]:
req = requests.get("https://www.filmweb.pl/Wladca.Pierscieni.Druzyna.Pierscienia")
soup = BeautifulSoup(req.text, 'html.parser')
filmweb.cell(row=1,column=1).value = 'Tytuł'
filmweb.cell(row=2,column=1).value = 'Reżyser'
filmweb.cell(row=3,column=1).value = 'Premiera'
filmweb.cell(row=4,column=1).value = 'Boxoffice'
filmweb.cell(row=5,column=1).value = 'Ocena'

title = filmweb.cell(row=1,column=2).value =soup.find('h1',itemprop="name").find('a').text
direction = filmweb.cell(row=2,column=2).value = soup.find(text="reżyseria").next_element.find("a").text
premiere = filmweb.cell(row=3,column=2).value = soup.find(text="premiera").next_element.find("a").text
boxoffice = soup.find(text="boxoffice").next_element.text
boxoffice = filmweb.cell(row=4,column=2).value = boxoffice.replace('top #27','')
rate = filmweb.cell(row=5,column=2).value = float(soup.find("span", itemprop="ratingValue").text.replace(",", "."))

print(f"tytuł: {title},\nocena: {rate},\nreżyseria: {direction},\npremiera: {premiere},\nboxoffice: {boxoffice}")
wb.save('Lab4_Murawski.xlsx')

tytuł: Władca Pierścieni: Drużyna Pierścienia,
ocena: 8.0,
reżyseria:   Peter Jackson ,
premiera:   10 grudnia 2001 (świat)  ,
boxoffice: $871 530 324 na świecie$315 544 750 w USA$555 985 574 poza USA
